In [ ]:
import re
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
import time
from tqdm import tqdm
import csv
import shutil

In [ ]:
# Path to the ZIP file containing the genomes to be extracted
zip_path = '/Path/to/downloaded/zipfiles.zip'

# Directory where the extracted files will be placed
extract_dir = 'extracted_genomes'

# Check if the target extraction directory already exists
if os.path.exists(extract_dir):
    # If it exists, then delete it plus its contents for a clean extraction
    shutil.rmtree(extract_dir)

# Create a new extraction directory
os.makedirs(extract_dir)

# Open the ZIP file and extract its contents to the specified directory
with zipfile.ZipFile(zip_path, 'r') as myzip:
    myzip.extractall(extract_dir)

# List all files and directories in the main extraction directory
extracted_data = os.listdir(extract_dir)

# Define the expected paths for specific subdirectories
ncbi_dataset_dir = os.path.join(extract_dir, 'ncbi_dataset')  # Directory where the dataset is expected
ncbi_data_dir = os.path.join(ncbi_dataset_dir, 'data')        # Subdirectory containing the main data

# Check if the 'data' directory exists and is indeed a directory
if os.path.exists(ncbi_data_dir) and os.path.isdir(ncbi_data_dir):
    # If the directory exists, update the extracted data to reflect its contents
    extracted_data = os.listdir(ncbi_data_dir)
else:
    # If the expected directory structure is missing, print a warning message
    print(f"Expected 'ncbi_dataset' directory not found in {extract_dir}")

In [ ]:
sequence_data = []
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(".json"):
            json_file_path = os.path.join(root, file)
            os.remove(json_file_path)

# Create an empty list to store sequence data (used later on)
sequence_data = []

# Define the directory where processed data will be stored
data_dir = 'data'

# Create the `data` directory if it doesn't already exist
os.makedirs(data_dir, exist_ok=True)

# Walk through all directories and files starting from the `extract_dir`
for root, dirs, files in os.walk(extract_dir):
    # Iterate over each file in the current directory
    for file in files:
        # Check if the file has a `.json` extension
        if file.endswith(".json"):
            # Get the full path to the JSON file
            json_file_path = os.path.join(root, file)
            # Remove the JSON file from the directory
            os.remove(json_file_path)

In [ ]:
# Loop through each item (file or folder) in the `extracted_data` list
for item in extracted_data:
    # Create the full path of the item
    item_path = os.path.join(ncbi_data_dir, item)
    
    # Check if the item is a directory
    if os.path.isdir(item_path):
        # List all files in the current folder
        files_in_folder = os.listdir(item_path)
        
        # Loop through each file in the folder
        for filename in files_in_folder:
            # Get the full file path
            file_path = os.path.join(item_path, filename)
            
            # Check if the file is a nucleotide sequence file in FASTA format
            if filename.endswith(".fna") or filename.endswith(".fasta"):
                # Open the file for reading
                with open(file_path, "r") as file:
                    # Parse the FASTA file and extract sequence records
                    for record in SeqIO.parse(file, "fasta"):
                        # Append the sequence ID and sequence to the `sequence_data` list
                        sequence_data.append({
                            'Sequence_ID': record.id,
                            'Sequence': str(record.seq)
                        })
            
            # Check if the file is a protein sequence file in FASTA format
            elif filename.endswith(".faa"):
                # Open the file for reading
                with open(file_path, "r") as file:
                    # Parse the FASTA file and extract protein sequence records
                    for record in SeqIO.parse(file, "fasta"):
                        # Append the sequence ID and sequence to the `sequence_data` list
                        sequence_data.append({
                            'Sequence_ID': record.id,
                            'Sequence': str(record.seq)
                        })
            
            # Check if the file is a GenBank file
            elif filename.endswith(".gb"):
                # Open the file for reading
                with open(file_path, "r") as file:
                    # Parse the GenBank file and extract the sequence record
                    gb_record = SeqIO.read(file, "genbank")
                    # Append the sequence ID and sequence to the `sequence_data` list
                    sequence_data.append({
                        'Sequence_ID': gb_record.id,
                        'Sequence': str(gb_record.seq)
                    })
# If the `ncbi_dataset` directory is not found, print an error message
else:
    print(f"Expected 'ncbi_dataset' directory not found in {ncbi_data_dir}")

In [ ]:
# Define the column names for the dataframe
# This step is to create a dataframe but a text file can be created if preferred
columns = ['Sequence_ID', 'Sequence']

# Create a pandas DataFrame from the `sequence_data` list with th columns 'Sequence_ID' and 'Sequence'
genome_data = pd.DataFrame(sequence_data, columns=columns)

# Define the path where the CSV file will be saved
csv_path = 'Genome_Name.csv'

# Save the DataFrame as a CSV file
# The `index=False` argument ensures that the row index is not included in the CSV
genome_data.to_csv(csv_path, index=False)

In [ ]:
sequence_column = genome_data['Sequence']
def smallest_rotation(ssr):
    return min(ssr[i:] + ssr[:i] for i in range(len(ssr)))

ssr_pattern = r'(\w+?)\1{4,50}'

def process_chunk(sequence_chunk, start_pos, seen_positions):
    chunk_ssrs = []

    for match in re.finditer(ssr_pattern, sequence_chunk):
        ssr = match.group(0)  # The matched SSR sequence
        ssr_start = start_pos + match.start()  # Absolute position of the SSR start
        ssr_end = ssr_start + len(ssr) - 1  # Absolute position of the SSR end
        
        if not any(start <= ssr_start <= end or start <= ssr_end <= end for start, end in seen_positions):
            # Add the SSR and its position if no overlap
            chunk_ssrs.append((ssr, ssr_start))
            seen_positions.append((ssr_start, ssr_end))  # Track the SSR position

    return chunk_ssrs

# Extract the 'Sequence' column from the DataFrame
sequence_column = genome_data['Sequence']

# Define a function to compute the smallest rotation of a string
def smallest_rotation(ssr):
    # Generate all rotations of the string and return the smallest one to avoid counting same SSR more than once
    return min(ssr[i:] + ssr[:i] for i in range(len(ssr)))

# Define a regular expression pattern to match SSRs
# This pattern looks for a repeating substring (\w+?) repeated 4 to 50 times consecutively
ssr_pattern = r'(\w+?)\1{4,50}'

# Function to process a chunk of a sequence and extract unique SSRs
# The chunk is to speed up the process however if computer is able to process large amounts of data this is not necessary
def process_chunk(sequence_chunk, start_pos, seen_positions):
    
    # Create a list to store SSRs and their positions in the current chunk
    chunk_ssrs = []

    # Use regular expressions to find all matches for the SSR pattern in the sequence chunk
    for match in re.finditer(ssr_pattern, sequence_chunk):
        ssr = match.group(0)  # Extract the matched SSR sequence
        ssr_start = start_pos + match.start()  # Calculate the absolute start position of the SSR
        ssr_end = ssr_start + len(ssr) - 1  # Calculate the absolute end position of the SSR

        # Check if this SSR overlaps with any previously seen SSR
        if not any(start <= ssr_start <= end or start <= ssr_end <= end for start, end in seen_positions):
            # If no overlap, add the SSR and its starting position to the results
            chunk_ssrs.append((ssr, ssr_start))
            # Record this SSR's position to track overlaps
            seen_positions.append((ssr_start, ssr_end))

    # Return the list of unique SSRs and their positions from this chunk
    return chunk_ssrs

In [ ]:
# Get the first genome sequence from the DataFrame's 'Sequence' column
genome_sequence = genome_data['Sequence'].iloc[0]

# Define the size of each sequence chunk to process and the step size for sliding the window
chunk_size = 10000  # Number of bases in each chunk, adjust according to own circumstances
step_size = 100    # Step size for sliding the window

# Total number of sequences in the column (the total number of SSRs)
total_sequences = len(sequence_column)

# Total number of bases in the selected genome sequence (for calculating SSR density)
total_bases = len(genome_sequence)

# Initialize an empty dictionary to store unique SSRs and their positions
unique_ssrs = {}

# Define a function to find SSRs using a sliding window approach
def sliding_window_ssr_finder(genome_sequence, chunk_size, step_size):

    total_bases = len(genome_sequence)  # Total length of the genome sequence
    seen_positions = []  # List to track positions of already processed SSRs

    # Iterate over the genome sequence using a sliding window approach
    for start in tqdm(range(0, total_bases - chunk_size + 1, step_size), desc="Finding SSRs"):
        # Extract a chunk of the sequence based on the current window
        sequence_chunk = genome_sequence[start:start + chunk_size]

        # Process the chunk to find SSRs and their positions
        chunk_ssrs = process_chunk(sequence_chunk, start, seen_positions)

        # Store unique SSRs in the dictionary
        for ssr, pos in chunk_ssrs:
            if ssr not in unique_ssrs:
                # Add the SSR to the dictionary with its first occurrence position
                unique_ssrs[ssr] = pos

    # Return the dictionary of unique SSRs
    return unique_ssrs

In [ ]:
# Call the `sliding_window_ssr_finder` function to find unique SSRs in the genome sequence
# Produces loading bar to indicate progress
# genome_sequence: The entire genome sequence being analyzed
# chunk_size: The size of each sliding window or chunk to process
# step_size: The step size for the sliding window movement
unique_ssrs = sliding_window_ssr_finder(genome_sequence, chunk_size, step_size)

In [ ]:
# Open a text file in write mode to save the identified SSRs and their positions
# Replace 'Genome' with the actual genome
with open("Genome_ssrs_found.txt", "w") as file:
    # Iterate over each SSR and its position in the `unique_ssrs` dictionary
    for ssr, pos in unique_ssrs.items():
        # Write the SSR and its position to the file in a formatted string
        file.write(f"SSR: {ssr}, Position: {pos}\n")

In [ ]:
# Calculate the total number of bases in kilobases (kbp)
total_bases_kbp = total_bases / 1000  # Convert total bases from base pairs to kilobases

# Calculate the total number of unique SSRs identified
total_ssrs = len(unique_ssrs)

# Join all unique SSR sequences into a single string for calculating total SSR bases
ssr_string = ''.join(unique_ssrs)

# Calculate the total number of bases covered by all SSRs
total_ssr_bases = len(ssr_string)

# Calculate SSR density (total SSR bases per kilobase of genome sequence)
ssr_density = total_ssr_bases / total_bases_kbp

In [ ]:
# Check if the genome has any bases (total_bases > 0) before calculating percentages
if total_bases > 0:

    # Count the number of occurrences of each nucleotide (A, T, C, G) in the genome sequence
    count_A = genome_sequence.count('A')  # Count the number of 'A's
    count_T = genome_sequence.count('T')  # Count the number of 'T's
    count_C = genome_sequence.count('C')  # Count the number of 'C's
    count_G = genome_sequence.count('G')  # Count the number of 'G's
    
    # Calculate the percentage
    percentage_A = (count_A / total_bases) * 100  # Percentage of 'A'
    percentage_T = (count_T / total_bases) * 100  # Percentage of 'T'
    percentage_C = (count_C / total_bases) * 100  # Percentage of 'C'
    percentage_G = (count_G / total_bases) * 100  # Percentage of 'G'
    
    # Optional:
    # Print the percentages of each nucleotide (two decimal places)
    print(f"Percentage of A: {percentage_A: .2f}%")
    print(f"Percentage of T: {percentage_T: .2f}%")
    print(f"Percentage of C: {percentage_C: .2f}%")
    print(f"Percentage of G: {percentage_G: .2f}%")
else:
    # If no sequence data exists (total_bases <= 0), print a message
    print("No SSRs were found")

In [ ]:
# Check if there are any SSR bases to analyze (total_ssr_bases > 0)
if total_ssr_bases > 0:

    # Count the occurrences of each nucleotide (A, T, C, G) in the compiled SSR string
    ssr_count_A = ssr_string.count('A')  # Count the number of 'A's in the SSRs
    ssr_count_T = ssr_string.count('T')  # Count the number of 'T's in the SSRs
    ssr_count_C = ssr_string.count('C')  # Count the number of 'C's in the SSRs
    ssr_count_G = ssr_string.count('G')  # Count the number of 'G's in the SSRs
    
    # Calculate the percentage of each nucleotide in the SSR sequences
    ssr_percentage_A = (ssr_count_A / total_ssr_bases) * 100  # Percentage of 'A' in SSRs
    ssr_percentage_T = (ssr_count_T / total_ssr_bases) * 100  # Percentage of 'T' in SSRs
    ssr_percentage_C = (ssr_count_C / total_ssr_bases) * 100  # Percentage of 'C' in SSRs
    ssr_percentage_G = (ssr_count_G / total_ssr_bases) * 100  # Percentage of 'G' in SSRs
    
    # Optional:
    # Print the percentages of A, T, C, and G in the SSR sequences (rounded to two decimal places)
    print(f"Percentage of A: {ssr_percentage_A: .2f}%")
    print(f"Percentage of T: {ssr_percentage_T: .2f}%")
    print(f"Percentage of C: {ssr_percentage_C: .2f}%")
    print(f"Percentage of G: {ssr_percentage_G: .2f}%")
else:
    # If no SSRs were found (total_ssr_bases <= 0), print a message
    print("No SSRs were found")

In [ ]:
# Define the Genome 
Genome = 'Name of Genome'

In [ ]:
# Define the file path for the CSV where data will be saved
csv_path = "/path/to/saved/data.csv"

# Create a dictionary representing a new row of data to add to the CSV file
new_row = {
    
    "Genome": Genome,  # The genome name/ID

    "total_bases": total_bases,  # The total length of the genome sequence (in base pairs)

    "total_ssrs": total_ssrs,  # The count of unique SSR sequences identified

    "total_ssr_bases": total_ssr_bases,  # Total number of bases in the SSR sequences

    "ssr_density": ssr_density,  # The density of SSRs in the genome sequence (bases/kb)

    # The percentages of each nucleotide (A, T, C, G) in the genome sequence
    "%A in bases": percentage_A, 
    "%T in bases": percentage_T,
    "%C in bases": percentage_C, 
    "%G in bases": percentage_G, 

    # The percentages of each nucleotide (A, T, C, G) in the SSR sequences
    "%A in ssrs": ssr_percentage_A, 
    "%T in ssrs": ssr_percentage_T, 
    "%C in ssrs": ssr_percentage_C, 
    "%G in ssrs": ssr_percentage_G 
}

In [ ]:
# Read the existing CSV file into a DataFrame (df)
df = pd.read_csv(csv_path)

# Convert the new_row dictionary into a DataFrame so it can be added to the existing DataFrame
new_row_df = pd.DataFrame([new_row])  

# Add the new row DataFrame with the existing DataFrame (df), ignoring the index to avoid conflicts
df = pd.concat([df, new_row_df], ignore_index=True)

In [ ]:
# Save the updated DataFrame (df) to the CSV file
df.to_csv(csv_path, index=False)